In [1]:
import numpy as np
import matplotlib as plt
from PIL import Image
import keras

Using TensorFlow backend.


### 🐍mnistデータ読み込む🐍
- 読み込んだ後に画像の画素値を-1～1に正規化した

In [2]:
(X_train, _), (_, _) = keras.datasets.mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

img_shape = X_train[0].shape

batch_size = 128
training_ratio = 5

half_batch = int(batch_size / 2)

### 🐍z_dim(潜在変数の次元)を定義🐍
- この潜在変数の次元が小さすぎるとモード崩壊という、同じような数字ばかりが生成されてしまう現象が起きてしまう。Generator必死に訓練データをまねようとするが、潜在変数の次元が小さいため表現の幅を持つことができずに、特定の数字のみに集中してDiscriminatorを騙そうとする

In [3]:
z_dim = 5

### 🐍Generatorを設計🐍

In [4]:
noise_shape = (z_dim,)
generator = keras.models.Sequential()

generator.add(keras.layers.Dense(1024, input_shape=noise_shape))
generator.add(keras.layers.normalization.BatchNormalization())
generator.add(keras.layers.advanced_activations.LeakyReLU(0.2))
generator.add(keras.layers.Dense(128*7*7))
generator.add(keras.layers.normalization.BatchNormalization())
generator.add(keras.layers.advanced_activations.LeakyReLU(0.2))
generator.add(keras.layers.Reshape((7,7,128), input_shape=(128*7*7, )))
generator.add(keras.layers.UpSampling2D((2, 2)))
generator.add(keras.layers.Conv2D(64, (5, 5), padding='same'))
generator.add(keras.layers.normalization.BatchNormalization())
generator.add(keras.layers.advanced_activations.LeakyReLU(0.2))
generator.add(keras.layers.UpSampling2D((2, 2)))
generator.add(keras.layers.Conv2D(1, (5, 5), padding='same'))
generator.add(keras.layers.Activation('tanh'))

Instructions for updating:
Colocations handled automatically by placer.


### 🐍Discriminatorを設計🐍

In [5]:
discriminator = keras.models.Sequential()

discriminator.add(keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=img_shape))
discriminator.add(keras.layers.advanced_activations.LeakyReLU(0.2))
discriminator.add(keras.layers.Conv2D(128, (5, 5), strides=(2, 2)))
discriminator.add(keras.layers.advanced_activations.LeakyReLU(0.2))
discriminator.add(keras.layers.Flatten())
discriminator.add(keras.layers.Dense(1024))
discriminator.add(keras.layers.advanced_activations.LeakyReLU(0.2))
discriminator.add(keras.layers.Dropout(0.5))
discriminator.add(keras.layers.Dense(1))
discriminator.add(keras.layers.Activation('sigmoid'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### 🐍いったんここで、discriminatorの学習をFalseにする🐍

In [6]:
for layer in discriminator.layers:
    layer.trainable = False
discriminator.trainable = False

### 🐍Combinedを設計🐍

In [8]:
z = keras.layers.Input(shape=(z_dim, ))
img = generator(z)
valid = discriminator(img)
netG_model = keras.models.Model(z, valid)
loss = -1 * keras.backend.mean(valid)
training_updates_1 = keras.optimizers.Adam(lr=1e-4, beta_1=0.5, beta_2=0.9).get_updates(generator.trainable_weights, [], loss)

netG_train = keras.backend.function([z], [loss], training_updates_1)

### 🐍さっきFalseにしたやつ戻す🐍

In [9]:
for layer in discriminator.layers:
    layer.trainable = True
for layer in generator.layers:
    layer.trainable = False
discriminator.trainable = True
generator.trainable = False

### 🐍modelを作成する🐍
- Discriminatorには、画像を3枚入力する
    - 1つ目は本物(r-img)、2つ目はGeneratorが作ったやつ(f-img)、3つ目はその2つの画像間の任意の点の集合から作った画像（！？）(a-img)


In [10]:
# Generatorに入力する、潜在変数zを生成する
z = keras.layers.Input(shape=(z_dim,))

# Discriminatorに入力する画像を3つ用意する
r_img = keras.layers.Input(shape=(img_shape))
f_img = generator(z)
e_input = keras.backend.placeholder(shape=(None, 1,1,1))
a_img = keras.layers.Input(shape=(img_shape), tensor=e_input * r_img + (1-e_input) * f_img)

# Discriminatorの出力を定義
f_out = discriminator(f_img)
r_out = discriminator(r_img)
a_out = discriminator(a_img)

### 🐍損失関数を作成する🐍
- 定義に沿って損失関数を作成
![Loss Function](WGAN_loss_func.png)

In [11]:
weight_lam = 10

# Original critic loss : 第一項目
loss_real = keras.backend.mean(r_out) / batch_size
loss_fake = keras.backend.mean(f_out) / batch_size

# Our gradient penalty : 第二項目
grad_mixed = keras.backend.gradients(a_out, [a_img])[0]
norm_grad_mixed = keras.backend.sqrt(keras.backend.sum(keras.backend.square(grad_mixed), axis = [1, 2, 3]))
grad_penalty = keras.backend.mean(keras.backend.square(norm_grad_mixed - 1))

# 最終的な損失関数
loss = loss_fake - loss_real + weight_lam * grad_penalty

### 🐍Optimiserのインスタンス化🐍

In [12]:
training_updates_2 = keras.optimizers.Adam(lr=1e-4, beta_1=0.5, beta_2=0.9).get_updates(discriminator.trainable_weights, [], loss)

### 🐍入出力とoptimiserをfunction化してしまう🐍

In [13]:
netD_train = keras.backend.function([r_img, z, e_input], [loss_real, loss_fake], training_updates_2)

### 🐍学習の回数epochsを決定！🐍

In [14]:
epochs = 20000

### 🐍いざ学習させる🐍

In [15]:
g_loss_array = np.zeros(epochs)
d_loss_array = np.zeros(epochs)
d_accuracy_array = np.zeros(epochs)
d_predict_true_num_array = np.zeros(epochs)

for epoch in range(epochs):
    
    for j in range(training_ratio):
        
        # 以下、Discriminator学習
        # Generatorによる偽データの生成
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        gen_imgs = generator.predict(noise)

        # バッチサイズの半数を教師データからピックアップ
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        epsilon = np.random.uniform(size=(batch_size, 1, 1, 1))
        errD_real, errD_fake = netD_train([imgs, noise, epsilon])
        d_loss = errD_real - errD_fake
        
        # discriminatorを学習させる
        d_predict = discriminator.predict_classes(np.concatenate([gen_imgs,imgs]), verbose=0)
        d_predict = np.sum(d_predict)

    # 以下、Generator学習
    noise = np.random.normal(0, 1, (batch_size, z_dim))
    g_loss = netG_train([noise])
    
    if epoch % 50 == 0:
        gen_imgs = np.squeeze(gen_imgs)
        save_img = gen_imgs[0] * 127.5 + 127.5
        Image.fromarray(save_img.astype(np.uint8)).save('savefig_WGAN/{}.png'.format(str(epoch)))
        print ("%d [D loss: %f] [G loss: %f]" % (epoch, d_loss, g_loss[0]))

0 [D loss: 0.000117] [G loss: -0.495896]
50 [D loss: -0.001436] [G loss: -0.682931]
100 [D loss: -0.002885] [G loss: -0.714871]
150 [D loss: -0.001335] [G loss: -0.659887]
200 [D loss: -0.001048] [G loss: -0.591204]
250 [D loss: -0.001593] [G loss: -0.478272]
300 [D loss: -0.000766] [G loss: -0.578372]
350 [D loss: -0.000633] [G loss: -0.527020]
400 [D loss: -0.000245] [G loss: -0.469541]
450 [D loss: -0.001148] [G loss: -0.684954]
500 [D loss: -0.000226] [G loss: -0.466877]
550 [D loss: 0.000221] [G loss: -0.575768]
600 [D loss: -0.000328] [G loss: -0.505321]
650 [D loss: -0.000284] [G loss: -0.529199]
700 [D loss: 0.000086] [G loss: -0.499736]
750 [D loss: -0.000097] [G loss: -0.509077]
800 [D loss: 0.000510] [G loss: -0.374557]
850 [D loss: 0.000301] [G loss: -0.465974]
900 [D loss: 0.000262] [G loss: -0.454587]
950 [D loss: 0.000131] [G loss: -0.482647]
1000 [D loss: 0.000769] [G loss: -0.437109]
1050 [D loss: 0.000112] [G loss: -0.504364]
1100 [D loss: 0.000648] [G loss: -0.463482

KeyboardInterrupt: 